In [1]:
# !pip install tensorflow
# !pip install transformers

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

# !pip install kaggle
# !mkdir -p ~/.kaggle
# !cp "/content/drive/My Drive/kaggle.json" ~/.kaggle
# !chmod 600 ~/.kaggle/kaggle.json


In [3]:
# # !kaggle datasets download -d elvis23/mental-health-conversational-data
# !mkdir - p kaggle/input/mental-health-conversational-data
# !unzip ../data/mental-health-conversational-data.zip - d kaggle/input/mental-health-conversational-data/

# Reading Data

In [4]:
import json
import numpy as np
import pandas as pd

with open(r'kaggle/input/mental-health-conversational-data/intents.json') as f:
    data = json.load(f)

df = pd.DataFrame(data['intents'])

df.head()


,tag,patterns,responses
0,greeting,"[Hi, Hey, Is anyone there?, Hi there, Hello, H...",[Hello there. Tell me how are you feeling toda...
1,morning,[Good morning],[Good morning. I hope you had a good night's s...
2,afternoon,[Good afternoon],[Good afternoon. How is your day going?]
3,evening,[Good evening],[Good evening. How has your day been?]
4,night,[Good night],"[Good night. Get some proper sleep, Good night..."


Each tag contain multiple questions & answers so i want to sprate them 

In [5]:
dic = {"tag": [], "patterns": [], "responses": []}
for i in range(len(df)):
    ptrns = df[df.index == i]['patterns'].values[0]
    rspns = df[df.index == i]['responses'].values[0]
    tag = df[df.index == i]['tag'].values[0]
    for j in range(len(ptrns)):
        dic['tag'].append(tag)
        dic['patterns'].append(ptrns[j])
        dic['responses'].append(rspns)

In [6]:
df = pd.DataFrame.from_dict(dic)

df.head()

,tag,patterns,responses
0,greeting,Hi,[Hello there. Tell me how are you feeling toda...
1,greeting,Hey,[Hello there. Tell me how are you feeling toda...
2,greeting,Is anyone there?,[Hello there. Tell me how are you feeling toda...
3,greeting,Hi there,[Hello there. Tell me how are you feeling toda...
4,greeting,Hello,[Hello there. Tell me how are you feeling toda...


# Data Preprocessing

In [7]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(lower=True, split=' ')
tokenizer.fit_on_texts(df['patterns'])
tokenizer.get_config()

vacab_size = len(tokenizer.word_index)
print('number of unique words = ', vacab_size)


ptrn2seq = tokenizer.texts_to_sequences(df['patterns'])
X = pad_sequences(ptrn2seq, padding='post')
print('X shape = ', X.shape)

lbl_enc = LabelEncoder()
y = lbl_enc.fit_transform(df['tag'])
print('y shape = ', y.shape)
print('num of classes = ', len(np.unique(y)))

2023-06-14 19:43:35.008280: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 19:43:37.152449: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-14 19:43:37.153478: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-14 19:43:41.039363: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


number of unique words =  303
X shape =  (232, 18)
y shape =  (232,)
num of classes =  80


Now i am going to build and train model


In [8]:
import tensorflow
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Input(shape=(X.shape[1])),
    keras.layers.Embedding(input_dim=vacab_size+1,
                           output_dim=100, mask_zero=True),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LayerNormalization(),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LayerNormalization(),
    keras.layers.LSTM(32),
    keras.layers.LayerNormalization(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.LayerNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.LayerNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(len(np.unique(y)), activation="softmax")
])

model.compile(optimizer='adam',
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 18, 100)           30400     
                                                                 
 lstm (LSTM)                 (None, 18, 32)            17024     
                                                                 
 layer_normalization (LayerN  (None, 18, 32)           64        
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 18, 32)            8320      
                                                                 
 layer_normalization_1 (Laye  (None, 18, 32)           64        
 rNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 32)                8

In [9]:
history = model.fit(
    x=X,
    y=y,
    batch_size=10,
    callbacks=[tensorflow.keras.callbacks.EarlyStopping(
        monitor='accuracy', patience=3)],
    epochs=30
)

Epoch 1/30
24/24 [==============================] - 11s 27ms/step - loss: 4.6829 - accuracy: 0.0302
Epoch 2/30
24/24 [==============================] - 1s 25ms/step - loss: 3.6341 - accuracy: 0.1466
Epoch 3/30
24/24 [==============================] - 1s 25ms/step - loss: 3.0343 - accuracy: 0.2931
Epoch 4/30
24/24 [==============================] - 1s 25ms/step - loss: 2.4784 - accuracy: 0.3879
Epoch 5/30
24/24 [==============================] - 1s 26ms/step - loss: 2.1973 - accuracy: 0.4526
Epoch 6/30
24/24 [==============================] - 1s 25ms/step - loss: 1.7943 - accuracy: 0.5819
Epoch 7/30
24/24 [==============================] - 1s 27ms/step - loss: 1.4707 - accuracy: 0.6509
Epoch 8/30
24/24 [==============================] - 1s 26ms/step - loss: 1.2402 - accuracy: 0.7069
Epoch 9/30
24/24 [==============================] - 1s 25ms/step - loss: 1.0402 - accuracy: 0.8017
Epoch 10/30
24/24 [==============================] - 1s 30ms/step - loss: 0.8923 - accuracy: 0.7888
Epoch 11

In [10]:
import re
import random


def model_responce(query):
    text = []
    txt = re.sub('[^a-zA-Z\']', ' ', query)
    txt = txt.lower()
    txt = txt.split()
    txt = " ".join(txt)
    text.append(txt)

    x_test = tokenizer.texts_to_sequences(text)
    x_test = np.ravel(x_test)
    
    x_test = pad_sequences([x_test], padding='post', maxlen=X.shape[1])
    y_pred = model.predict(x_test, verbose=0)
    y_pred = y_pred.argmax()
    tag = lbl_enc.inverse_transform([y_pred])[0]
    responses = df[df['tag'] == tag]['responses'].values[0]

    print("you: {}".format(query))
    print("model: {}".format(random.choice(responses)))


In [11]:
while True:
    input_text = input("you: ")
    if input_text == 'exit':
        break
    answer = model_responce(input_text)

you: Hello
model: Hello there. Glad to see you're back. What's going on in your world right now?
you: Who are you?
model: You can call me SootheBot.
you: Am I okay?
model: For a diagnosis of depression, a person needs to have experienced low mood or loss of interest or pleasure in life for at least 2 weeks. Also, they will have experienced the following symptoms: feelings of sadness, hopelessness, or irritability nearly every day.
you: I don't feel okay
model: Don't be hard on yourself. What's the reason behind this?


In [12]:
import tensorflow as tf

# saved_model format (pb)
model.save('mental_chatbot')

2023-06-14 19:45:25.257648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-14 19:45:25.259608: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-14 19:45:25.260802: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: mental_chatbot/assets


INFO:tensorflow:Assets written to: mental_chatbot/assets


In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

2023-06-14 19:50:16.528896: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-14 19:50:16.530645: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-14 19:50:16.531791: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /tmp/tmpddyfcxff/assets


INFO:tensorflow:Assets written to: /tmp/tmpddyfcxff/assets
2023-06-14 19:50:52.161229: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-14 19:50:52.161382: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-14 19:50:52.161461: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error 

ConverterError: /home/codespace/.python/current/lib/python3.10/site-packages/tensorflow/python/saved_model/save.py:1276:0: error: 'tf.TensorListReserve' op requires element_shape to be static during TF Lite transformation pass
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
/home/codespace/.python/current/lib/python3.10/site-packages/tensorflow/python/saved_model/save.py:1276:0: error: failed to legalize operation 'tf.TensorListReserve' that was explicitly marked illegal
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall"]): called from
<unknown>:0: error: Lowering tensor list ops is failed. Please consider using Select TF ops and disabling `_experimental_lower_tensor_list_ops` flag in the TFLite converter object. For example, converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]\n converter._experimental_lower_tensor_list_ops = False


In [16]:
import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model('model.h5')

# Convert the Keras model to a TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

2023-06-14 19:54:57.439037: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-14 19:54:57.440783: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-14 19:54:57.442232: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /tmp/tmpp615h134/assets


INFO:tensorflow:Assets written to: /tmp/tmpp615h134/assets
2023-06-14 19:55:33.211406: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2023-06-14 19:55:33.211553: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2023-06-14 19:55:33.211634: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error 